In [1]:
import requests
import json
import csv
import time
import os.path
import pandas as pd

In [2]:
'''getQuery function takes a url request and header dict, then returns a formatted json file'''
def getQuery(url, headers):
    # Get get request for json 
    r = requests.get(url, headers=headers)
    # use json loader to format output
    json_data = json.loads(r.text)
    # return the data and when that data was queried
    return json_data

'''formatQuery takes a json_data set then formats it according to the runeliteAPI timeseries data then returns the results'''
def formatQuery(json_data, itemName):
    itemID = json_data['itemId']
    samples = []
    # Loop through data key from queried data
    for item in json_data['data']:
        entry = (
            itemID,
            itemName,
            item['timestamp'],
            item['avgHighPrice'],
            item['avgLowPrice'],
            item['highPriceVolume'],
            item['lowPriceVolume']
        )
        samples.append(entry)
    # Return
    return samples
def runeliteToCSV(samples, filePath, colNames, appendIfPossible=True): 
    # Check if we need to append or write
    # Append if path exists and append is on
    if os.path.isfile(filePath) and appendIfPossible:
        writeMode = 'a'
        msg = "appended"
    else:
        writeMode = 'w'
        msg = "written"
        
    with open(filePath, mode=writeMode, newline='') as f:
        # Create csv writer
        csvWriter = csv.writer(f, delimiter = ',')
        # Write column names if not appending
        if writeMode == 'w':
            csvWriter.writerow(colNames)
            
        # Write out item data
        for entry in samples:
            csvWriter.writerow(entry)
    print(f"Output {msg} to {filePath}")
             

In [3]:
'''Define needed variables'''
# API key used to access summary jsons
runeliteAPI = "https://prices.runescape.wiki/api/v1/osrs/"

# Example exestion for time series query: /timeseries?timestep=5m&id=4151
itemName = "Ranger_boots"
timeSteps = ["5m", "1h", "24h"]

# Get id based off item name
df = pd.read_csv("data/item_keys.csv")
itemID = int(df[df['name'] == itemName]['id'].iloc[0])
print(f"Item ID found: {itemID}")

# User agent tag used to let wiki know why I am using their API. 
userAgent = {
    'User-Agent': 'Machine Learning Practice',
    'From': 'sliverwall (discord ID)'  # discord ID
}

# Define headers for output file
colNames = ["itemID", "name", "timestamp","avgHighPrice", "avgLowPrice", "highPriceVolume", "lowPriceVolume"]

Item ID found: 2577


In [4]:
'''Output section. Runs main logic'''
for timeStep in timeSteps:
    runeliteAPIQuery = runeliteAPI + f"timeseries?timestep={timeStep}&id={itemID}"

    print(f"Query being used: {runeliteAPIQuery}")

    # Define where data will be written to
    fileName = f"runelite_data_{timeStep}_{itemID}"
    outputDir = "data/runelite/"
    fileExe = ".csv"
    filePath = outputDir + fileName + fileExe

    # Collect data
    data = getQuery(url=runeliteAPIQuery,
                    headers=userAgent)

    # Format data
    samples = formatQuery(json_data=data, itemName=itemName)

    # Write data to output file
    runeliteToCSV(samples=samples, filePath=filePath, colNames=colNames, appendIfPossible=False) 
    print("Wait a moment before using the API again...")
    if timeStep != timeSteps[-1]:
        time.sleep(5)

Query being used: https://prices.runescape.wiki/api/v1/osrs/timeseries?timestep=5m&id=2577
Output written to data/runelite/runelite_data_5m_2577.csv
Wait a moment before using the API again...
Query being used: https://prices.runescape.wiki/api/v1/osrs/timeseries?timestep=1h&id=2577
Output written to data/runelite/runelite_data_1h_2577.csv
Wait a moment before using the API again...
